This code is based on an assignment from the DeepLearning.AI's Coursera course titled Sequence Models. I have expanded on the assignment by writing two models with different architectures and writing a helper function that facilitates model evaluation.

**Skills demonstrated:** LSTMs, attention, RNNs, NLP, tensorflow, data processing, model evaluation, general coding

Overview: 

> Long-Short Term Memory (LSTM) networks are powerful tools for natural language processing (NLP) tasks. They expand on basic recurrent neural network (RNN) models by encoding a "memory" that addresses the vanishing gradient problem that persists in classic RNNs. Attention models expand on traditional RNNs by passing more information between encoder and decoder layers, thereby allowing the model to learn important features connecting parts of text that appear more than a few words apart. This code contains three LSTM models: one without attention and a more traditional architecture, one with attention, and one with a naive approximation to attention that simply uses fully connected layers to pass information between the encoder and decoder layers.

Models:

> 1. LSTM without attention. Encoder layer is a bidirectional LSTM. Output from the last timestep is fed into a (unidirectional) LSTM decoder layer.
2. LSTM with attention. Encoder layer is a bidirectional LSTM. Output <i> from each time step </i> **and** the hidden state from the previous time step of the decoder layer is fed into an attention layer, which passes the LSTM output and hidden state through a dense layer, then through a softmax layer that is used to calculate the context. The context is then fed into the decoder layer, which is a (unidirectional) LSTM.
3. LSTM with approximated attention. Similar to the LSTM model with attention, but the attention layer does not receive the hidden state from the decoder layer and passes the output of the dense layer directly to the decoder layer.

Purpose: 
> Demonstrate the effectiveness of LSTMs with attention performing NLP tasks.

Datasets: 

> A randomly generated dataset containing a prescribed number of dates in a variety of human-readable formats

Target:

> The given date in machine-readable format. (Assumptions: If no day is specified in the human-readable date, assign the day as the first of the month. If no month is specified, assign January.)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import time

from faker import Faker
import random
from tqdm import tqdm
#from babel.dates import format_date
from date_std_tools import *

In [2]:
#Generate data
m = 20000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|███████████████████████████████████| 20000/20000 [00:03<00:00, 6034.71it/s]


In [3]:
#dataset[:50]
#print('Number of characters in human_vocab: ' + str(len(human_vocab)))
#print('Number of characters in machine_vocab: ' + str(len(machine_vocab)))
#print('Characters in human_vocab: ' + str([x[0] for x in list(human_vocab.items())]))
#print('Characters in machine_vocab: ' + str([x[0] for x in list(machine_vocab.items())]))

In [4]:
#Define the max length of human-readable dates (Tx) and machine readable dates (Ty)
#In the context of the model, Tx is the max character length of an input string and Ty is the length of the output
Tx = 20
Ty = 10

#X (Y): lists of length Tx representing each human-readable (machine-readable) date. Dimension (m,Tx) ((m,Ty))
#Xoh (Yoh): one-hot representation of X (Y). Dimension (m,Tx,len(human_vocab)) ((m,Ty,len(machine_vocab)))
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (20000, 20)
Y.shape: (20000, 10)
Xoh.shape: (20000, 20, 37)
Yoh.shape: (20000, 10, 11)


In [28]:
#Define LSTM layer sizes
n_a = 32 # number of units for the bi-directional LSTM's hidden state a
n_s = 64 # number of units for the uni-directional LSTM's hidden state s
num_epochs = 1

In [6]:
# Define shared layers as global variables for model_with_attention
repeat = RepeatVector(Tx)
concat = Concatenate(axis=-1)
dense1 = Dense(10, activation = "relu")
dense2 = Dense(1, activation = "relu")
activate = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dot = Dot(axes = 1)

post_attention_LSTM = LSTM(n_s, return_state = True)
softmax_layer_attn = Dense(len(machine_vocab), activation=softmax)

model_attn = model_with_attention(Tx, Ty, n_a, n_s,
                                  len(human_vocab), len(machine_vocab),
                                  repeat, concat, dense1, dense2, activate, dot,
                                  post_attention_LSTM, softmax_layer_attn)
opt = Adam(learning_rate=0.005, weight_decay=0.005) # Adam(...) 
model_attn.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [29]:
start_attn = time.time()
model_attn.fit([Xoh, s0, c0], outputs, epochs=num_epochs, batch_size=64)
end_attn = time.time()

313/313 [==============================] - 19s 60ms/step - loss: 0.2753 - dense_2_loss: 0.0169 - dense_2_1_loss: 0.0170 - dense_2_2_loss: 0.0678 - dense_2_3_loss: 0.1474 - dense_2_4_loss: 1.8870e-04 - dense_2_5_loss: 0.0012 - dense_2_6_loss: 0.0023 - dense_2_7_loss: 3.0350e-04 - dense_2_8_loss: 0.0093 - dense_2_9_loss: 0.0129 - dense_2_accuracy: 0.9912 - dense_2_1_accuracy: 0.9911 - dense_2_2_accuracy: 0.9707 - dense_2_3_accuracy: 0.9524 - dense_2_4_accuracy: 1.0000 - dense_2_5_accuracy: 0.9999 - dense_2_6_accuracy: 1.0000 - dense_2_7_accuracy: 1.0000 - dense_2_8_accuracy: 0.9979 - dense_2_9_accuracy: 0.9970


In [8]:
#model_attn.summary()

In [9]:
#Define shared LSTM layer for model_approx_attention
LSTM_approx = LSTM(units=n_s, return_state = True)
softmax_layer_approx = Dense(len(machine_vocab), activation=softmax)

model_approx_attn = model_approx_attention(Tx, Ty, n_a, n_s,
                                           len(human_vocab), len(machine_vocab),
                                           LSTM_approx, softmax_layer_approx)

opt = Adam(learning_rate=0.005, weight_decay=0.005) # Adam(...) 
model_approx_attn.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [30]:
start_approx = time.time()
model_approx_attn.fit([Xoh, s0, c0], outputs, epochs=num_epochs, batch_size=64)
end_approx = time.time()

313/313 [==============================] - 43s 136ms/step - loss: 0.5171 - dense_3_loss: 0.0191 - dense_3_1_loss: 0.0189 - dense_3_2_loss: 0.0822 - dense_3_3_loss: 0.1922 - dense_3_4_loss: 1.3641e-04 - dense_3_5_loss: 0.0153 - dense_3_6_loss: 0.0825 - dense_3_7_loss: 2.7081e-04 - dense_3_8_loss: 0.0566 - dense_3_9_loss: 0.0500 - dense_3_accuracy: 0.9916 - dense_3_1_accuracy: 0.9909 - dense_3_2_accuracy: 0.9668 - dense_3_3_accuracy: 0.9391 - dense_3_4_accuracy: 1.0000 - dense_3_5_accuracy: 0.9947 - dense_3_6_accuracy: 0.9756 - dense_3_7_accuracy: 1.0000 - dense_3_8_accuracy: 0.9816 - dense_3_9_accuracy: 0.9865


In [11]:
#model_approx_attn.summary()

In [12]:
#Define shared LSTM layer for model_just_LSTM
LSTM_just = LSTM(units=n_s, return_state = True)
softmax_layer_just = Dense(len(machine_vocab), activation=softmax)

model_just_LSTM = model_just_LSTM(Tx, Ty, n_a, n_s,
                                  len(human_vocab), len(machine_vocab),
                                  LSTM_just, softmax_layer_just) 

opt = Adam(learning_rate=0.005, weight_decay=0.005) # Adam(...) 
model_just_LSTM.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [31]:
start_just = time.time()
model_just_LSTM.fit([Xoh, s0, c0], outputs, epochs=num_epochs, batch_size=64)
end_just = time.time()

313/313 [==============================] - 8s 27ms/step - loss: 1.0958 - dense_14_loss: 0.0258 - dense_14_1_loss: 0.0237 - dense_14_2_loss: 0.1484 - dense_14_3_loss: 0.2743 - dense_14_4_loss: 4.0678e-04 - dense_14_5_loss: 0.0265 - dense_14_6_loss: 0.1001 - dense_14_7_loss: 0.0010 - dense_14_8_loss: 0.2398 - dense_14_9_loss: 0.2558 - dense_14_accuracy: 0.9900 - dense_14_1_accuracy: 0.9901 - dense_14_2_accuracy: 0.9505 - dense_14_3_accuracy: 0.9233 - dense_14_4_accuracy: 1.0000 - dense_14_5_accuracy: 0.9920 - dense_14_6_accuracy: 0.9765 - dense_14_7_accuracy: 0.9999 - dense_14_8_accuracy: 0.9091 - dense_14_9_accuracy: 0.9082


In [14]:
#model_just_LSTM.summary()

In [15]:
#model.save_weights("LSTM_model.weights.h5")
#Load pre-trained params if not fitting
#model.load_weights("LSTM_model.weights.h5")

In [16]:
#load list of human-readable dates from dates.txt
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

doc = [x.lower().strip().replace(',','').replace('.','') for x in doc]

In [17]:
acc_attn, wrong_attn, _ = model_performance_test(model_attn, doc, human_vocab, inv_machine_vocab, Tx, n_s)
print('LSTM with attention:')
print('Time to train ' + str(num_epochs) + ' epochs : ' + f"{end_attn-start_attn:.2f}" + ' seconds.')
print('Accuracy: ' + str(acc_attn))
print('Number of dates incorrectly predicted: ' + str(len(wrong_attn)))
print('Incorrectly predicted dates in the form (Input date, predicted date, actual date):\n' + str(wrong_attn))

16/16 [==============================] - 4s 9ms/step
LSTM with attention:
Time to train 3 epochs : 109.63 seconds.
Accuracy: 0.996
Number of dates incorrectly predicted: 2
Incorrectly predicted dates in the form (Input date, predicted date, actual date):
[('10/04/1998', '1998-10-01', '1998-10-04'), ('november 2004', '2004-11-00', '2004-11-01')]


In [18]:
acc_approx, wrong_approx, _ = model_performance_test(model_approx_attn, doc, human_vocab, inv_machine_vocab, Tx, n_s)
print('LSTM with approximated attention:')
print('Time to train ' + str(num_epochs) + ' epochs: ' + f"{end_approx-start_approx:.2f}" + ' seconds.')
print('Accuracy: ' + str(acc_approx))
print('Number of dates incorrectly predicted: ' + str(len(wrong_approx)))
print('Incorrectly predicted dates in the form (Input date, predicted date, actual date):\n' + str(wrong_approx))

16/16 [==============================] - 5s 23ms/step
LSTM with approximated attention:
Time to train 3 epochs: 156.54 seconds.
Accuracy: 0.966
Number of dates incorrectly predicted: 17
Incorrectly predicted dates in the form (Input date, predicted date, actual date):
[('5/21/1977', '1977-04-21', '1977-05-21'), ('6/24/1987', '1987-05-24', '1987-06-24'), ('12/01/1973', '1973-12-00', '1973-12-01'), ('01 oct 1979', '1979-10-30', '1979-10-01'), ('04 dec 1988', '1988-02-04', '1988-12-04'), ('december 23 1999', '1999-02-23', '1999-12-23'), ('dec 2009', '2009-02-01', '2009-12-01'), ('june 1999', '1999-01-01', '1999-06-01'), ('12/2005', '2005-02-01', '2005-12-01'), ('12/2008', '2008-02-01', '2008-12-01'), ('12/2012', '2012-02-01', '2012-12-01'), ('12/2014', '2014-02-01', '2014-12-01'), ('12/1994', '1994-02-01', '1994-12-01'), ('12/1986', '1986-02-01', '1986-12-01'), ('12/2007', '2007-02-01', '2007-12-01'), ('12/1989', '1989-02-01', '1989-12-01'), ('12/2004', '2004-02-01', '2004-12-01')]


In [19]:
acc_just, wrong_just, _ = model_performance_test(model_just_LSTM, doc, human_vocab, inv_machine_vocab, Tx, n_s)
print('LSTM model, no attention:')
print('Time to train ' + str(num_epochs) + 'epochs : ' + f"{end_just-start_just:.2f}" + ' seconds.')
print('Accuracy: ' + str(acc_just))
print('Number of dates incorrectly predicted: ' + str(len(wrong_just)))
print('Sample incorrectly predicted dates in the form (Input date, predicted date, actual date):\n' + str(wrong_just[:10]))

16/16 [==============================] - 4s 5ms/step
LSTM model, no attention:
Time to train 3epochs : 51.82 seconds.
Accuracy: 0.764
Number of dates incorrectly predicted: 118
Sample incorrectly predicted dates in the form (Input date, predicted date, actual date):
[('7/8/1971', '1981-07-08', '1971-07-08'), ('2/6/1996', '1996-02-26', '1996-02-06'), ('4/10/1971', '1971-04-11', '1971-04-10'), ('4/09/1975', '1975-04-04', '1975-04-09'), ('8/01/1998', '1998-08-11', '1998-08-01'), ('1/25/2011', '2011-11-25', '2011-01-25'), ('4/12/1982', '1982-04-11', '1982-04-12'), ('10/13/1976', '1976-10-33', '1976-10-13'), ('7/21/1998', '1998-07-11', '1998-07-21'), ('10/21/1979', '1979-10-11', '1979-10-21')]


In [20]:
#acc, wrong = model_performance_test(model_attn, doc, human_vocab, Tx)

Next we will use mispelled dates to test how robust the models are to spelling errors

In [35]:
doc_misspell = ['decembre 3, 2000', 'merch 1986', 'juen, 17, 1999', '29 fbreuary 2024', 'novmebr 35 20100']

In [36]:
m_pred = len(doc_misspell)
s00 = np.zeros((m_pred, n_s))
c00 = np.zeros((m_pred, n_s))
_, Xoh_pred = prepare_input(doc_misspell, human_vocab, Tx)
    
pred_attn = model_attn.predict([Xoh_pred, s00, c00])
pred_approx = model_approx_attn.predict([Xoh_pred, s00, c00])
pred_just = model_just_LSTM.predict([Xoh_pred, s00, c00])
    
#convert output of the model into a list of human-readable dates
output_attn = prepare_output(pred_attn, inv_machine_vocab)
output_approx = prepare_output(pred_approx, inv_machine_vocab)
output_just = prepare_output(pred_just, inv_machine_vocab)


#Create a list of 4-tuples of (human_readable, model_attn_predicted, 
# model_approx_attn_predicted, model_just_LSTMs_predicted)
input_output = list(zip(doc_misspell, output_attn, output_approx, output_just))

print('LSTM with attention:')
print('Predicted dates in the form (Input date, model_attn predicted date, model_approx_attn predicted, model_just_LSTMs predicted):\n')
for x in input_output:
    print(str(x)+'\n')

1/1 [==============================] - 0s 23ms/step
LSTM with attention:
Predicted dates in the form (Input date, model_attn predicted date, model_approx_attn predicted, model_just_LSTMs predicted):

('decembre 3, 2000', '2000-12-03', '2000-12-03', '2000-12-03')

('merch 1986', '1986-03-01', '1986-03-01', '1986-03-01')

('juen, 17, 1999', '1999-06-17', '1999-06-17', '1999-07-17')

('29 fbreuary 2024', '2024-02-29', '2024-02-29', '2024-02-09')

('novmebr 35 20100', '2010-11-05', '2000-11-33', '2000-11-35')



The model with attention was best able to standardize each of the above misspelled dates among the three models.